In [1]:
import psycopg2

conn = psycopg2.connect(
    host = 'rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net',
    user = 'test_user',
    password = 'j2M{CnnFq@',
    database = 'db1',
    port = '6432'
)

cur = conn.cursor()

In [ ]:
cur.execute('''
-- Таблица нужна для последующей очистки. 
-- "created_at" конвертирован в timestamptz во избежании последующих расчетных ошибок (часовой пояс выбран исходя из распределения кол-ва сообщений по часам, с учетом локального времени)
WITH filthy_query AS (
    SELECT 
        message_id,
        entity_id,
        type,
        created_by,
        to_timestamp(created_at) AT TIME ZONE 'UTC-3' AS created_at,
        LAG(type) OVER (PARTITION BY entity_id ORDER BY created_at ASC) AS type_perv
    FROM test.chat_messages
),

-- Подчищенная таблица (предполагается, что будут удалены все последовательные сообщения от одного отправителя)
clear_query AS (
    SELECT
        message_id,
        entity_id,
        type,
        created_by,
        created_at AS outcome_dt,
        LAG(created_at) OVER (PARTITION BY entity_id ORDER BY created_at ASC) AS income_dt
    FROM filthy_query 
    WHERE (type != type_perv OR type_perv IS NULL)
),

prefinally_query AS (
    SELECT
        *,
        CASE
            -- Написали в нерабочее, ответили в рабочее *считаем рабочие минуты.
            WHEN income_dt::time < '9:30'::time AND outcome_dt::time  >= '9:30'::time
                THEN outcome_dt - (income_dt::date + interval '570 min')
                
            -- Написали в нерабочее, ответили в нерабочее *округляем до начала(исх.)/конца(вход.) рабочего дня в случаях, если сообшения пришли в разные дни. Иначе - timedelta = 0
            WHEN income_dt::time < '9:30'::time AND outcome_dt::time < '9:30'::time
                THEN CASE 
                    WHEN income_dt::date != outcome_dt::date
                        THEN outcome_dt::date - (income_dt::date + interval '570 min')
                    ELSE '0 seconds'::interval
                END
            --Написали в рабочее, ответили в нерабочее * считаем только рабочие минуты (округляем исходящее)
            WHEN income_dt::time >= '9:30'::time AND outcome_dt::time < '9:30'::time
                THEN outcome_dt::date - income_dt
            ELSE outcome_dt - income_dt
        END messages_timediff
    FROM clear_query
    WHERE type = 'outgoing_chat_message'
),

finally_query AS (
SELECT
    a.*,
    -- Вычитаем нерабочее время в случаях, когда ответ занял более суток с учетом предшествующих округлений
    EXTRACT(EPOCH FROM a.messages_timediff) - ((EXTRACT(EPOCH FROM a.messages_timediff)/86400)::integer * 570*60) AS timediff_clear,
    CASE 
        WHEN b.name_mop IS NULL
            THEN 'Неидентифицированный Пользователь'
        ELSE b.name_mop
    END manager,
    CASE 
        WHEN c.rop_name IS NULL 
            THEN 'Неизвестный'
        ELSE replace(c.rop_name, ' РОП', '')
    END rop
FROM prefinally_query a
LEFT JOIN test.managers b
ON a.created_by = b.mop_id
LEFT JOIN test.rops c
ON b.rop_id::integer = c.rop_id
WHERE income_dt IS NOT NULL
)

SELECT manager, ROUND(AVG(timediff_clear),2)
FROM finally_query
GROUP BY manager
;''')

res = cur.fetchall()
for row in res:
    print(row)  
    




In [ ]:
import pandas as pd

cur.execute('''SELECT * FROM test.chat_messages''')
chat_messages = pd.DataFrame(cur.fetchall(), columns=['message_id', 'type', 'entity_id', 'created_by', 'created_at'])
cur.execute('''SELECT * FROM test.rops''')
rops = pd.DataFrame(cur.fetchall(), columns=['rop_id', 'rop_name'])
cur.execute('''SELECT * FROM test.managers''')
managers = pd.DataFrame(cur.fetchall(), columns=['mop_id', 'name_mop', 'rop_id'])
managers['rop_id'] = managers['rop_id'].astype(int)


# Добавление временных данных из столбца created_at
chat_messages['created_at'] = pd.to_datetime(chat_messages['created_at']+(3*60*60), unit='s')

# filthy_query
chat_messages['type_perv'] = chat_messages.sort_values(['entity_id', 'created_at']) \
                                          .groupby('entity_id')['type'] \
                                          .shift(1)

# clear_query
clear_query = chat_messages[
    (chat_messages['type'] != chat_messages['type_perv']) | chat_messages['type_perv'].isna()
]
clear_query = clear_query.sort_values(['entity_id', 'created_at'])
clear_query['income_dt'] = clear_query.groupby('entity_id')['created_at'].shift(1)
clear_query['outcome_dt'] = clear_query['created_at']

# prefinally_query
def calculate_messages_timediff(row):
    # Временные метки
    start_time = row['income_dt']
    end_time = row['outcome_dt']
    if pd.isna(start_time) or pd.isna(end_time):
        return pd.Timedelta(seconds=0)

    # Рабочее время
    work_start = pd.Timestamp(start_time.date()) + pd.Timedelta(minutes=9*60+30)
    work_end = pd.Timestamp(start_time.date()) + pd.Timedelta(hours=18)

    if start_time.time() < work_start.time() and end_time.time() >= work_start.time():
        # Написали в нерабочее, ответили в рабочее
        return end_time - work_start
    elif start_time.time() < work_start.time() and end_time.time() < work_start.time():
        # Написали и ответили в нерабочее
        if start_time.date() != end_time.date():
            return pd.Timestamp(end_time.date()) - start_time
        else:
            return pd.Timedelta(seconds=0)
    elif start_time.time() >= work_start.time() and end_time.time() < work_start.time():
        # Написали в рабочее, ответили в нерабочее
        return pd.Timestamp(end_time.date()) - start_time
    else:
        # Обычное время
        return end_time - start_time

clear_query['messages_timediff'] = clear_query.apply(calculate_messages_timediff, axis=1)

# Фильтруем только "outgoing_chat_message"
prefinally_query = clear_query[clear_query['type'] == 'outgoing_chat_message']

# finally_query
def adjust_for_non_working_time(timediff):
    seconds_in_day = 86400
    work_seconds = 570 * 60
    total_seconds = timediff.total_seconds()
    days = total_seconds // seconds_in_day
    return total_seconds - (days * work_seconds)

prefinally_query['timediff_clear'] = prefinally_query['messages_timediff'].apply(adjust_for_non_working_time)

# Присоединяем таблицы managers и rops
prefinally_query = prefinally_query.merge(managers, left_on='created_by', right_on='mop_id', how='left')
prefinally_query = prefinally_query.merge(rops, left_on='rop_id', right_on='rop_id', how='left')

# Добавляем столбцы manager и rop
prefinally_query['manager'] = prefinally_query['name_mop'].fillna('Неидентифицированный Пользователь')
prefinally_query['rop'] = prefinally_query['rop_name'].fillna('Неизвестный').str.replace(' РОП', '')

# Итоговая агрегация
result = prefinally_query.groupby('manager')['timediff_clear'].mean().round(2).reset_index()

# Вывод
result